Assignment 3 : Classification Finale
===
##Hector Curi - Spencer Kaiser

---
Instructions
---
You are to build upon the predictive analysis (classification) that you already completed in assignment two, adding additional modeling from new classification algorithms. You should use appropriate cross validation for all of your analysis (you should exlpain your chosen method of validation). Try to use as much testing data as possible in a realistic manner (you should define what you think is realistic and why). Follow the CRISP-DM framework in your analysis (you are not performing all of the CRISP-DM outline, only the portions relevant to the grading rubric outlined below). This report is worth 20% of the final grade. You may complete this assignment in teams of as many as three people (but no more). Only turn in one report to blackboard. 

Write a report covering all the steps of the project. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

---
Dataset Selection
---
Select a dataset identically to the way you selected for assignment one. You are not required to use the same dataset that you used for assignment one, but you are encouraged. You must identify two or more classification tasks from the dataset. For example, if your dataset was from the diabetes data you might try to predict two tasks: (1) if a patient will be readmitted within a 30 day period or not, and (2) what the total number of days a patient will spend in the hospital, given their history and specifics of the encounter. 

---
### Choice of Classification Tasks
We want to predict the following:
1. Predict if a person makes more than $50,000 dollars a year, based on their age, race, sex, etc. (all of the attributes)
2. Predict if a person makes capital trades, based on their age, race, sex, etc. (all of the attributes)
3. Predict if a person is male, based on their workclass, occupation, age, etc. (all of the attributes)
---
### Initial Code


In [ ]:
#import all packages used in this assignment
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pandas.tools.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from numpy import random as rd
from sklearn import metrics as mt
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

# read in the csv file
df_raw = pd.read_csv('data/adult.data')

Data Preparation
---

---
### 1. Class Variables (10 points)
**Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis**

In [ ]:
# Here is the general description of our data
df_raw.info()

In [ ]:
# As we can see, it seems that all of the values are there, but after looking at the actual data file
# we noticed that there are some values recorded as '?', these are the missing values.

# First let's replace ' ?' with NaN (not a number) for all the missing values
df = df_raw.replace(to_replace='?',value=np.nan)

# Now let's see how many values we actually have
df.info()

In [ ]:
# We can see that workclass, occupation and native-country have some missing values.
# Because only 2399 rows have missing values in them out of 32561 total (7.36% of the data),
# We will just remove those instances from our dataset.

df = df.dropna(how='any')

# Now the data has 30162 instances with no missing values. :)
df.info()

In [ ]:
# Now we are going to remove the attributes that we think are not useful for our model
# fnlwgt is an assigned weight by the census bureau, we don't know it's formula so its useless to us
del df['fnlwgt']

# education-num is an ordinal representation of the education level, which is better for data modelling
# thus we remove the education attribute
del df['education']

# relationship contains very strange values such as "Not-in-family, Other-relative, Own-child", so we will get rid of it.
del df['relationship']

In [ ]:
# Now our data contains these attributes
df.info()

In [ ]:
# Now we are going to rename the attributes for an easier understanding of the data
df = df.rename(columns={'income': 'salaryOver50K', 'sex': 'isMale', 'education-num':'educationLevel',
                        'marital-status':'maritalStatus','capital-gain':'capitalGain','capital-loss':'capitalLoss',
                        'native-country':'nativeCountry','hours-per-week':'hoursPerWeek'})

In [ ]:
# Now our data contains these attributes
df.info()

In [ ]:
# Now let's convert to camel case all of the values with dashes in workclass
df = df.replace(to_replace='Federal-gov',value='FederalGovernment')
df = df.replace(to_replace='State-gov',value='StateGovernment')
df = df.replace(to_replace='Never-worked',value='NeverWorked')
df = df.replace(to_replace='Private',value='Private')
df = df.replace(to_replace='Self-emp-inc',value='SelfEmployedIncorporated')
df = df.replace(to_replace='Self-emp-not-inc',value='SelfEmpoyedNotIncorporated')
df = df.replace(to_replace='Local-gov',value='LocalGovernment')
df = df.replace(to_replace='Without-pay',value='WithoutPay')

# Now let's convert to camel case all of the values with dashes in marital status
df = df.replace(to_replace='Married-AF-spouse',value='MarriedAFSpouse')
df = df.replace(to_replace='Married-civ-spouse',value='MarriedCivSpouse')
df = df.replace(to_replace='Married-spouse-absent',value='MarriedSpouseAbsent')
df = df.replace(to_replace='Never-married',value='NeverMarried')

# Now let's convert to camel case all of the values with dashes in occupation
df = df.replace(to_replace='Adm-clerical',value='AdmClerical')
df = df.replace(to_replace='Armed-Forces,',value='ArmedForces,')
df = df.replace(to_replace='Craft-repair',value='CraftRepair')
df = df.replace(to_replace='Exec-managerial',value='ExecManagerial')
df = df.replace(to_replace='Farming-fishing',value='FarmingFishing')
df = df.replace(to_replace='Handlers-cleaners',value='HandlersCleaners')
df = df.replace(to_replace='Machine-op-inspct',value='MachineOpsInspector')
df = df.replace(to_replace='Other-service',value='OtherService')
df = df.replace(to_replace='Priv-house-serv',value='PrivHouseServ')
df = df.replace(to_replace='Prof-specialty',value='ProfSpecialty')
df = df.replace(to_replace='Protective-serv',value='ProtectiveServant')
df = df.replace(to_replace='Tech-support',value='TechSupport')
df = df.replace(to_replace='Transport-moving',value='TransportMoving')

# Now let's convert to camel case all of the values with dashes in race
df = df.replace(to_replace='Amer-Indian-Eskimo',value='AmerIndianEskimo')
df = df.replace(to_replace='Asian-Pac-Islander',value='AsianPacIslander')

# Now let's convert to camel case all of the values with dashes in nativeCountry
df = df.replace(to_replace='Dominican-Republic',value='DominicanRepublic,')
df = df.replace(to_replace='El-Salvador',value='ElSalvador,')
df = df.replace(to_replace='Holand-Netherlands',value='Netherlands')
df = df.replace(to_replace='Outlying-US(Guam-USVI-etc)',value='OutlyingUS')
df = df.replace(to_replace='Puerto-Rico',value='PuertoRico')
df = df.replace(to_replace='Trinadad&Tobago',value='Trinidad&Tobago')
df = df.replace(to_replace='United-States',value='US')

In [ ]:
# Now our data looks like this
df.head()

In [ ]:
# Now let's replace the binomial attributes with binary values 1 and 0
# replace sex values with binary values
df = df.replace(to_replace='Female',value=0)
df = df.replace(to_replace='Male',value=1)

# replace income values with binary values
df = df.replace(to_replace='<=50K',value=0)
df = df.replace(to_replace='>50K',value=1)

In [ ]:
# Now our data looks like this
df.head()

In [ ]:
# Now let's rearrange the columns
df = df[['isMale','age','race','nativeCountry','maritalStatus',
         'educationLevel','workclass','occupation','hoursPerWeek',
         'capitalGain','capitalLoss','salaryOver50K']]
df.head()

In [ ]:
# We can also group by age range like this
df['ageRange'] = pd.cut(df.age,[16,20,30,40,50,60,70,80,91],8,
                         labels=['17-20','20s','30s','40s','50s','60s','70s','80s',])

In [ ]:
# And by hours-per-week
df['hourRange'] = pd.cut(df['hoursPerWeek'], [0,21,41,61,81,100], 5, labels=['1-20', '21-40', '41-60','61-80','80-99'])

In [ ]:
# For capital gain and capital loss, based on the fact that someone with a capital gain
# can't have a capital loss, and viceversa, we will combine the attributes into one
# then we will store as a boolean of wheter they make capital transactions or not
# most people in the dataset have a value of 0 in both, so it is more useful to know 
# wheter they invest in capital or not, as opposed to how much they do it
df['hasCapitalTransactions'] = ((df.capitalGain - df.capitalLoss)!=0).astype(np.int)

In [ ]:
df.head()

In [ ]:
# Also here are the data types of our attributes, as we can see isMale and salaryOver50K are already integers
df.info()

In [ ]:
# Now we will make all the categorical (object) attributes into one-hot encoded values
# Also store current dataframe just in case
dfClean = df.copy(deep=True)

# race
tmp_df = pd.get_dummies(df.race,prefix='race')
dfModel = pd.concat((df,tmp_df),axis=1)

# nativeCountry
tmp_df = pd.get_dummies(df.nativeCountry,prefix='origin')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# maritalStatus
tmp_df = pd.get_dummies(df.maritalStatus,prefix='marriage')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# workclass
tmp_df = pd.get_dummies(df.workclass,prefix='workclass')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# occupation
tmp_df = pd.get_dummies(df.occupation,prefix='occupation')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# occupation
tmp_df = pd.get_dummies(df.ageRange,prefix='ageRange')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# occupation
tmp_df = pd.get_dummies(df.hourRange,prefix='hourRange')
dfModel = pd.concat((dfModel,tmp_df),axis=1)

# print data model
df = dfModel
df.info()

In [ ]:
df.head()

In [ ]:
# Finally, let's delete the object attributes from our data model
del df['race']
del df['nativeCountry']
del df['maritalStatus']
del df['occupation']
del df['workclass']
del df['capitalGain']
del df['capitalLoss']
del df['age']
del df['hoursPerWeek']
del df['ageRange']
del df['hourRange']

---
### 2. Final Data Set (5 points)
**Describe the final dataset that is used for classification (include a description of any newly formed variables you created).**

In [ ]:
# Now our data set looks like this
df.info(20)
df.head()

The final data set contains 'complete' instances which have no missing values. This data set is roughly 93% of the original data set and consists of binary and one-hot encoded attributes, with one attribute remaining as ordinal (education). We decided to create a variety of new attributes to better analyze their impact upon the target attribute, 'salaryOver50K'. 

'salaryOver50K' is a binary variable which indicates whether an individual, an instance in the context of the data set, has a salary over $50k. 

Because of the minimal number of instances in which individuals have either capital gains or losses, we decided to create a boolean representation of whether or not the individual has any gains or losses whatsoever. If they do, 'hasCapitalTransactions' is set to a 1.

Furthermore, we decided to break up age and salary into categorical, one-hot encoded variables for analysis. Both of the attributes are broken into chunks that adequately break up the data for easy analysis.

Finally, we decided to convert sex to a binary value and to do so, we chose to label the attribute as 'isMale' for readability. This way, the attribute is a binary value, however, the values are still readable to someone viewing the data.

Modeling and Evaluation
---

---
### 1. Evaluation Metrics (10 points)
**Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or a mix of all of them). Why are these appropriate for analyzing the results of your modeling?**

<div style="color:red;">

As discussed in our previous lab, the primary use of the data gleaned from this analysis will be for marketing purposes (Classification Task 1). In addition, the analysis of whether an individual makes capital trades (Classificiation Task 2) may be very useful to the IRS for identifying individuals that could potentially attempt to evade their taxes by failing to report their capital earnings.
<br>
<br>
While it may not appear that that these classifications and associated real-world uses have much in common, they have one important commonality: the risk of omitting a potential lead/audit (false negative) greatly outweighs the risk of including an incorrectly classified individual (false positive). 
<br>
<br>
For example, if a marketer were to include someone who does not make a salary of over 50k (false positive) in a promotion targeted at wealthy individuals, it will likely not generate any sales but it will only incur a small cost. On the other hand, if the data does not include an individual that was classified incorrectly (false negative), there is a potential for a significant loss in sales, especially in terms of the promotional campaign as a whole.
<br>
<br>
In order to minimize the amount of false negatives generated by our model, we have decided to use <strong>Recall</strong> as our primary evaluation metric. For the purposes of this assignment, Recall evaluations will be conducted in the context of identifying >50k salaries (<i>i.e., if an individual falls on the border between target classes, classify as >50k</i>) and in the context of identifying individuals who make capital trades. We will also be utilizing <strong>Accuracy</strong> as a form of baseline accuracy and for additional analyses.
<br>
<br>

</div>

---
### 2. Training and Testing Split (10 points)
**Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.**

<div style="color:red;">

We will use <strong>Stratified 10-Fold Cross Validation</strong> for our analysis. Stratified 10-Fold Cross Validation will provide our analysis with a high degree of generalization and refinement, however, it is appropriate for use in our analysis due to one characteristic of our data set:

<ul>
<li>
<strong>Data Consistency (Locality)</strong>: Because the data in our data set was collected from across the United States and because it includes such a diverse sample of individuals, it is very likely that the features that are highly correlated to our target classes will be indicators of that target class <i>regardless</i> of the location from which we obtain additional sample data. In other words, the locality of our testing data should not influence the classification. This is important for future analysis using the model we create; if this model is tested against data from a specific locale in the future, under the assumption stated above, the model should still maintain a similar level of accuracy to that of the analysis of the entire data set. As a result, the number of instances per fold will not have any significant impact upon our analysis.
</li>
</ul>

<br>
<br>
Stratified 10-Fold Cross Validation will provide us with a thorough method of splitting, training, and testing our models against our data set.
</div>

---
### 3. Classification Models (20 points)
**Create three different classification/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the thrid could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance.**

First, let's make a copy of our final data set, just in case.

In [ ]:
dfFinal = df.copy(deep=True)

For training and testing purposes, let's gather the data we have and randomly grab 80% of the instances for training and the remining 20% for testing.
We will create a permutation of the indices to be used in all of the models.

In [ ]:
N = len(df) # total number of instances
permuted_indices = rd.permutation(N) # random permutation of the instances
N_eighty_percent = int(0.8*N)

### Classification Task 1
Predict if a person makes more than 50, 000 dollars a year, based on their age, race, sex, etc. (all of the attributes)

#### Prepare Variables

In [ ]:
# get the dataset
df1 = dfFinal.copy(deep=True)
y = df1['salaryOver50K'].values # get the labels
del df1['salaryOver50K']
X = df1.values

X_train = X[permuted_indices[:N_eighty_percent]] 
X_test = X[permuted_indices[N_eighty_percent:]]

y_train = y[permuted_indices[:N_eighty_percent]]
y_test = y[permuted_indices[N_eighty_percent:]]

# scale attributes by the training set to normalize data
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find points that make this zero mean and unit std
X_train = scl_obj.transform(X_train) # apply to training
X_test = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

print 'Training sizes, X:', X_train.shape, 'Y:', y_train.shape
print 'Testing sizes, X:', X_test.shape, 'Y:', y_test.shape

#### Logistic Regression

In [ ]:
# run logistic regression and vary some parameters
LRTask1 = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object
LRTask1.fit(X_train,y_train)  # train object

y_hat = LRTask1.predict(X_test) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelOneReg = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

#### Support Vector Machine

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels

# train the model just as before
svm_clf1 = SVC(C=1.0, kernel='linear', degree=3, gamma=0.0) # get object
svm_clf1.fit(X_train, y_train)  # train object

y_hat = svm_clf1.predict(X_test) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelOneSVM = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

### Classification Task 2
Predict if a person makes capital trades, based on their age, race, sex, etc. (all of the attributes)

#### Prepare Variables

In [ ]:
df2 = dfFinal.copy(deep=True)
y = df2['hasCapitalTransactions'].values # get the labels
del df2['hasCapitalTransactions']
X = df2.values

X_train = X[permuted_indices[:N_eighty_percent]] 
X_test = X[permuted_indices[N_eighty_percent:]]

y_train = y[permuted_indices[:N_eighty_percent]]
y_test = y[permuted_indices[N_eighty_percent:]]

# scale attributes by the training set to normalize data
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find points that make this zero mean and unit std
X_train = scl_obj.transform(X_train) # apply to training
X_test = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

print 'Training sizes, X:', X_train.shape, 'Y:', y_train.shape
print 'Testing sizes, X:', X_test.shape, 'Y:', y_test.shape

#### Logistic Regression

In [ ]:
# run logistic regression and vary some parameters

LRTask2 = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object
LRTask2.fit(X_train,y_train)  # train object

y_hat = LRTask2.predict(X_test) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelTwoReg = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

#### Support Vector Machine

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels

# train the model just as before
svm_clf2 = SVC(C=1.0, kernel='linear', degree=3, gamma=0.0) # get object
svm_clf2.fit(X_train, y_train)  # train object

y_hat = svm_clf2.predict(X_test) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelTwoSVM = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

### Classification Task 3
Predict if a person is male, based on their workclass, occupation, age, etc. (all of the attributes)

#### Prepare Variables

In [ ]:
df3 = dfFinal.copy(deep=True)
y = df3['isMale'].values # get the labels
del df3['isMale']
X = df3.values

X_train = X[permuted_indices[:N_eighty_percent]] 
X_test = X[permuted_indices[N_eighty_percent:]]

y_train = y[permuted_indices[:N_eighty_percent]]
y_test = y[permuted_indices[N_eighty_percent:]]

# scale attributes by the training set to normalize data
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find points that make this zero mean and unit std
X_train = scl_obj.transform(X_train) # apply to training
X_test = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

print 'Training sizes, X:', X_train.shape, 'Y:', y_train.shape
print 'Testing sizes, X:', X_test.shape, 'Y:', y_test.shape

#### Logistic Regression

In [ ]:
# run logistic regression and vary some parameters

LRTask3 = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object
LRTask3.fit(X_train,y_train)  # train object

y_hat = LRTask3.predict(X_test) # get test set precitions

from sklearn import metrics as mt

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelThreeReg = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

#### Support Vector Machine

In [ ]:
# lets investigate SVMs on the data and play with the parameters and kernels

# train the model just as before
svm_clf3 = SVC(C=1.0, kernel='linear', degree=3, gamma=0.0) # get object
svm_clf3.fit(X_train, y_train)  # train object

y_hat = svm_clf3.predict(X_test) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
modelThreeSVM = acc
print("Accuracy of this model is: %.2f" % (acc*100))+"%"

---
### 4. Attribute Importance (10 points)
**Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes.**

---
### 5. Results Analysis with Evaluation Metrics (10 points)
**Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis.**

---
### 6. Model Advantages (10 points)
**Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods.**

Deployment
---

---
### 1. Evaluation Metrics (5 points)
**Build on and update your dicussion from last assignment: How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used? How would your deploy your model for interested parites? What other data should be collected? How often would the model need to be updated, etc.?**

Exceptional Work (10 points)
---

---
**You have free reign to provide additional modeling.**

### Old Stuff below this Line
---

---
### 2. Model Advantages (5 points)
**Discuss the advantages of each model for each classification task.**

For each of the classification tasks, both models are very accurate, with over 75% accuracy for the least accurate model.

The advantage of the Logistic Regression model is that it is much faster to run than the Support Vector Machine and gives a comparable accuracy.

In our computers the logistic regresion took at most 2 seconds, whereas the SVM took at least 30 seconds.

---
### 3. Logistic Regression Features (15 points)
**Use the weights from logistic regression to interpret the importance of different features for each classification task. Explain your interpretation.**

#### Classification Task 1
Predict if a person makes more than 50, 000 dollars a year, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# sort these attributes and spit them out
zip_vars = zip(LRTask1.coef_.T,df1.columns)
zip_vars.sort(key = lambda t: np.abs(t[0]))
for coef, name in reversed(zip_vars):
    print name, 'has importance of', coef[0]

In [ ]:
# now let's make a pandas Series with the names and values, and plot them
%matplotlib inline

weights = pd.Series(LRTask1.coef_[0],index=df1.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

As we can see from the bar chart above, the attributes that are most related to **making more than 50K a year** are:
* The Education Level
* Being Married to a civilian
* Having capital transactions

These make sense, especially capital transactions because if people that have enough money to be investing in stock, real state, etc, probably have a salary higher than 50K.
Being married to a civilian (the other choice was married to someone from the air force) also shows that a typical married life might make it easy to grow professionally and increase your salary.
Lastly, the education level, is directly related, simply because the higher the education level is, the more people will pay for your work.

On the opposite end of the bar chart, the inversely related attributes are:
* Being 17 to 20 years old
* Never being Married
* Having other service as occupation

The age is fairly obvious, because someone under 20 years old is unlikely to make more than 50K a year.
The interesting one is never being married, which seems to inversely affect your chances of making a high salary.
Finally, for the occupation as "other service", it probably refers to the service industry or jobs as such.

#### Classification Task 2
Predict if a person makes capital trades, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# sort these attributes and spit them out
zip_vars = zip(LRTask2.coef_.T,df2.columns)
zip_vars.sort(key = lambda t: np.abs(t[0]))
for coef, name in reversed(zip_vars):
    print name, 'has importance of', coef[0]

In [ ]:
# now let's make a pandas Series with the names and values, and plot them
%matplotlib inline

weights = pd.Series(LRTask2.coef_[0],index=df2.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

As we can see from the bar chart above, the attributes that are most related to **having capital trades are**:
* Having a salary over 50K
* Being in your 60s
* Education level

Similarly to our previous classification task, a salary larger than 50K is closely related to making investments and/or capital transactions. This is probably because of the sparse income that can be risked in an easier way.
Apparently people on their 60s tend to make a lot of capital trades, and also the education level is directly related.

On the opposite end of the bar chart, the inversely related attributes are:
* Originating from Scotland, Thailand or the Outlying US
* Having other service as occupation
* Being 17 to 20 years old

The age is fairly obvious, because someone under 20 years old is unlikely to make capital trades.
The other two are more ambiguous predictions.

#### Classification Task 3
Predict if a person is male, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# sort these attributes and spit them out
zip_vars = zip(LRTask3.coef_.T,df3.columns)
zip_vars.sort(key = lambda t: np.abs(t[0]))
for coef, name in reversed(zip_vars):
    print name, 'has importance of', coef[0]

In [ ]:
# now let's make a pandas Series with the names and values, and plot them
%matplotlib inline

weights = pd.Series(LRTask3.coef_[0],index=df3.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

As we can see from the bar chart above, the attributes that are most related to **being a male** are:
* Occupation in craft/repairs, transport/moving, handlers/cleaners and fishing/farming
* Being married to a civilian
* Working from 41-60 hours a week

We can see that those occupations are mostly associated with males, but being married and working 41-60 hours a week are hard to see the relationship.

On the opposite end of the bar chart, the inversely related attributes are:
* Being widowed or divorced or separated
* Occupation in Adm Clerical, Prof Specialty, Private Hous Service
* Working from 1-20 hours

Considering that this census is from 1994, these predictions seem to be based on the traditional household, where woman stayed at home or wokred part time.
It is interesting to see how being widowed, divorced or separated is mostly associated with women in this data set.

---
### 4. SVM Support Vectors (15 points)
**Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain.**

#### Classification Task 1
Predict if a person makes more than 50, 000 dollars a year, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# look at the support vectors
print svm_clf1.support_vectors_.shape
print svm_clf1.support_.shape
print svm_clf1.n_support_ 

In [ ]:
# These are the support vectors of our classification task 1
weights = pd.Series(svm_clf1.coef_[0],index=df1.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

The chart above indicates a variety of attributes that are either positively or negatively correlated with whether or not an individual has a salaray of **more than $50K**:
* Education level (highly positive correlation)
* Being married to a civilian spouse (highly positive correlation)
* Making capital transactions (either making gains or losses, both positive correlations)
* Having never been married before or remaining single after a divorce (both negative correlations)
* Being between the age of 17-20 (negative correlation)

These results are very understandable. It would be expected that someone who has a high education level, is married to a civilian spouse, and/or makes capital transactions would make a yearly salary of $50K or greater.

It is also not surprising that being young (17-20) or being single (either never married or single after a divorce) is correlated with having a salary of less than $50k per year.

#### Classification Task 2
Predict if a person makes capital trades, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# look at the support vectors
print svm_clf2.support_vectors_.shape
print svm_clf2.support_.shape
print svm_clf2.n_support_ 

In [ ]:
# These are the support vectors of our classification task 2
weights = pd.Series(svm_clf2.coef_[0],index=df2.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

The chart above indicates that there are several attributes which are highly correlated with **making capital transactions** are:
* Occupation as administration or clerical positions (negative correlation)
* Race of Asian/Pacific Islander (negative correlation)
* Hour Range (Either between 41-60 [negative correlation] or between 1 and 20 [positive correlation])
* US Origin (positive correlation)

These results are somewhat surprising, however, the origin and racial considerations may be rooted in cultural norms and life outlooks. The correlation between hours worked (1-20) and making capital transactions is somewhat surprising as many would imagine this correlation would be negative, however, it is positive in the context of our training set. Gathering additional information or additional instances of data may alter this attributes affect on the target.

#### Classification Task 3
Predict if a person is male, based on their age, race, sex, etc. (all of the attributes)

In [ ]:
# look at the support vectors
print svm_clf3.support_vectors_.shape
print svm_clf3.support_.shape
print svm_clf3.n_support_ 

In [ ]:
# These are the support vectors of our classification task 3
weights = pd.Series(svm_clf3.coef_[0],index=df3.columns)
weights = weights.order(ascending=True)
weights.plot(kind='barh',figsize=(8,20))

Finally, this chart indicates that there are several attributes present in the model for classification task 3 which are highly correlated with whether or not an individual is **male**:
* Being married to a civilian spouse (highly positive correlation)
* Having an occupation as a Craftsman or Repairman (highly positive correlation)
* Being self employed (positive correlation)
* Having never been married before or remaining single after a divorce (both negative correlations)

These results fall in line with expectations. The occupations discussed above would more than likely be dominantly male, being self employed would seem to be correlated with being male in the US, and finally, it is not surprising that males are also more likely to never become married or to remain single after becoming divorced.

---
### 4. Model Performace (5 points)
**Assess how well each model performs (use training/test data as appropriate)**

In [ ]:
# For each model, we used the same training/testing sizes:
# TRAINING: 80% of the data set
# TESTING: 20% of the data set

In [ ]:
print "MODEL 1 PERFORMANCE:"
print("Logistic Regression: %.2f" % (modelOneReg*100))+"%"
print("SVM: %.2f" % (modelOneSVM*100))+"%"+"\n"

In [ ]:
print "MODEL 2 PERFORMANCE:"
print("Logistic Regression: %.2f" % (modelTwoReg*100))+"%"
print("SVM: %.2f" % (modelTwoSVM*100))+"%"+"\n"

In [ ]:
print "MODEL 3 PERFORMANCE:"
print("Logistic Regression: %.2f" % (modelThreeReg*100))+"%"
print("SVM: %.2f" % (modelThreeSVM*100))+"%"+"\n"

As we can see from these numbers, both models perform excedingly well.
We highly prefer logistic regression because it runs extremely fast and produces very accurate predictions.

Evaluation and Deployment
---

---
### 1. Model Evaluation (5 points)
**How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used? How would your deploy your model for interested parites? What other data should be collected? How often would the model need to be updated, etc. ?**

Out of our three classification models, the most likely to be adopted by businesses would be the model that predicts whether an individual makes a salary of $50k or more. A person’s trading activity would be important information to certain businesses, especially in terms of identifying potential tax audits or for identifying discrepancies in reported earnings, however, it is unlikely that it would be widely adopted. Furthermore, the model that predicts whether an individual is male might be helpful in the presence of certain demographic information, however, it is highly likely that other data attributes would be a much stronger indicator of whether an individual is male.

The model predicting salary of an individual would be incredibly useful for businesses. This model could be used (assuming the business had access to demographic information) to help predict wether or not a specific individual would be interested in purchasing a luxury product or service, or at minimum, whether the person could actually afford this product or service. Given the high rate of accuracy of our model, it would be very beneficial to certain businesses.

In order to measure the model’s value, we could evaluate the conversion rate from targeted sales efforts. If the model identifies a certain portion of the population that is likely to make over $50k, targeted advertisements could be made to those individuals. The rate of which those ads lead to sales could be used as an indicator for the overall value of the model and the use of the model.

Deploying the model would require a decent amount of data regarding the individual(s) being targeted. Once this data was obtained, it would be very simple to identify the which members of the population are estimated to make a salary of $50k or more and advertisements could then be made to these individuals. After advertisements have reached their targets, data should be recorded as to whether or not the individual purchased the product or service. This data could eventually be used to form a new model which further predicts whether an individual would be interested in a specific product or service as opposed to a category of products or services.

Lastly, the model would not need to be updated, however, additional information may help further refine the model and improve accuracy. In addition, the information mentioned above could be used to form a new model which may be even more beneficial to specific businesses.

Exceptional Work (10 points)
---

---
### 1. Model Evaluation (5 points)
**You have free reign to provide additional modeling. One idea: fuse the outputs of the two classifiers to create a new classifier or use grid searching to estimate parameters of the models.**

We did 3 classification tasks, instead of only 2, which counts for additional modeling and exceptional work. :D